OPEN CHARGE MAP

    - API doc : https://openchargemap.org/site/develop/api#/

OPEN STREET MAP:

    - osmr :   > http://project-osrm.org/
               > https://www.jawg.io/docs/apidocs/routing/osrm/
               > http://project-osrm.org/docs/v5.24.0/api/?language=Python#important-interface-objects

    - openrouteservice.org:
        > lib : >https://github.com/GIScience/openrouteservice-py
        > https://classic-maps.openrouteservice.org/directions?n1=48.69096&n2=4.707642&n3=8&b=0&c=0&k1=en-US&k2=km
        > https://maps.openrouteservice.org/#/place/@8.76709,49.510944,10
    
    - osm-python-tools
        > https://github.com/mocnik-science/osm-python-tools/blob/master/docs/element.md

    - Nominatim
        DEBUG PAGE >  https://nominatim.openstreetmap.org/ui/search.html    
        API  >  https://nominatim.org/release-docs/develop/api/Overview/
    
    - EXTRACTIONS:
        OSM planet  >  https://planet.openstreetmap.org/
        Extracts    >  GeoFabrik.de : > https://download.geofabrik.de/openstreetmap/
                    >  BBBike.org :   > https://download.bbbike.org/osm/
                        L> region extract : > https://extract.bbbike.org/
                        L> country region extract from:
                                Garmin      : > https://download3.bbbike.org/osm/garmin/region/
                                mapsforge   : > https://download3.bbbike.org/osm/mapsforge/region/

TOMTOM:

    - ev routing
    - ev routing long range : https://developer.tomtom.com/routing-api/documentation/extended-routing/long-distance-ev-routing
    - ev ReachableRange                              

[+] Objectif:
    - Tester différents parcours entre deux villes
    - Trouver les zones sans bornes qui font échouer runs

[+] Méthodo:
    - differents types de VE
    - max 6 trajets alternatifs
    - Résultats:
        * Réussi
        * Echec -> ajout au catalogue de la zone où une borne aurait été nécessaire

[+] Distances:
    - Euclidienne
    - distance Matrix by tomtom semble utilisabe              

In [1]:
import requests, json, os, datetime

os.environ['TOMTOM_TOKEN'] = 'yjFXQX1HCdQjGQAmSOgf013L8bwaqify'
data_path = 'data'

print(f'requests version: {requests.__version__}')
print(f'json version: {json.__version__}')

requests version: 2.28.1
json version: 2.0.9


In [2]:
session = requests.Session()
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
session.headers.update(headers)

def get_(url):
    response = session.get(url.strip())
    return response.json()
# GET COORDINATES FROM CITY NAME

def get_city_coords(city, country):
    url  =  'https://nominatim.openstreetmap.org/search?'
    # url += f'q={adress}%2C+'
    url += f'city={city}&'
    url += f'country={country}&'

    url +=  'format=geojson&polygon_geojson=1'
    return get_(url)
    
   
'''https://gist.github.com/wassname/9f08c8a1c7f6d7fd377069b56f603e09'''
import shapely.geometry
def geojson2shp(geojson):
    """Convert geometry of any geojson to a shapely object."""
    if geojson['type'] == 'Feature':
        return geojson2shp(geojson["geometry"])
    elif geojson['type'] == 'FeatureCollection':
        geometries = [geojson2shp(geom) for geom in geojson['features']]
        return shapely.geometry.GeometryCollection(geometries)
    elif geojson['type'] == 'Point':
        return shapely.geometry.Point(geojson['coordinates'])
    elif geojson['type'] == 'MultiPoint':
        points = [
            geojson2shp(dict(type='Point', coordinates=poly))
            for poly in geojson['coordinates']
        ]
        return shapely.geometry.MultiPoint(points)
    elif geojson['type'] == 'LineString':
        return shapely.geometry.LineString(geojson['coordinates'])
    elif geojson['type'] == 'MultiLineString':
        linestrings = [
            geojson2shp(dict(type='LineString', coordinates=poly))
            for poly in geojson['coordinates']
        ]
        return shapely.geometry.MultiLineString(linestrings)
    elif geojson['type'] == 'Polygon':
        return shapely.geometry.Polygon(geojson['coordinates'][0])
    elif geojson['type'] == 'MultiPolygon':
        polygons = [
            geojson2shp(dict(type='Polygon', coordinates=poly))
            for poly in geojson['coordinates']
        ]
        return shapely.geometry.MultiPolygon(polygons)
    elif geojson['type'] == 'GeometryCollection':
        geometries = [geojson2shp(geom) for geom in geojson['geometries']]
        return shapely.geometry.GeometryCollection(geometries)
    else:
        raise Exception('Unknown geojson type: %s' % geojson)
    

def tomtom2linestring(feature):
    return LineString([ [dct['longitude'], dct['latitude']] for dct in feature ])

def concatenate_legs(legs):
    points = []
    for leg in legs:
        points.extend(leg['points'])
    return points 
       

In [3]:
from shapely import Polygon, MultiPolygon

class City:
    def __init__(self, feature_collection):
        self.type =feature_collection['features'][0]['type']
        self.properties = feature_collection['features'][0]['properties']
        self.name = self.properties['display_name'].split(', ')[0]
        self.bbox = feature_collection['features'][0]['bbox']
        self.area = geojson2shp(feature_collection['features'][0]['geometry'])
        # self.contour = ['coordinates'][0]
        # self.polygon = Polygon([(p[0], p[1]) for p in self.contour])
        self.point = self.area.centroid
        

In [4]:
# TODO : change route aquisition to get alternatives

# route_ORSR = Route(start, goal)('ORS')

start = City(get_city_coords('MARSEILLE','France'))
goal = City(get_city_coords('LILLE','France'))
start.point, goal.point

(<POINT (5.419 43.292)>, <POINT (3.047 50.632)>)

In [5]:
from shapely import LineString

class RouteORS:
    def __init__(self, feature_collection, start_point, goal_point):
        self.start_point = start_point
        self.goal_point = goal_point
        self.api = 'open route service'
        self.bbox = feature_collection['features'][0]['bbox']
        self.properties = feature_collection['features'][0]['properties']
        self.distance = self.properties['summary']['distance']
        self.duration = self.properties['summary']['duration']
        self.trace = geojson2shp(feature_collection['features'][0]['geometry'])

class RouteTT:
    def __init__(self, feature_collection, start_point, goal_point):
        self.start_point = start_point
        self.goal_point = goal_point
        self.api = 'tomtom'
        self.bbox = None
        self.properties = feature_collection['summary']
        self.distance = self.properties['lengthInMeters']
        self.duration = self.properties['travelTimeInSeconds']
        self.legs = feature_collection['legs']
        self.trace = tomtom2linestring(concatenate_legs(self.legs))

class RouteOSE:
    def __init__(self, feature_collection, start_point, goal_point):
        self.start_point = start_point
        self.goal_point = goal_point
        self.api = 'open service route map'
        self.bbox = None
        self.properties = 'fastest'
        self.distance = feature_collection['routes'][0]['distance']
        self.duration = feature_collection['routes'][0]['duration']
        self.trace = geojson2shp(feature_collection['routes'][0]['geometry'])

            

def request_route_tt_api(start,goal, car):
    token = os.environ.get('TOMTOM_TOKEN')
    url = "https://api.tomtom.com/routing/1/calculateRoute/"
    url += f"{start.point.y}%2C{start.point.x}%3A"
    url += f"{goal.point.y}%2C{goal.point.x}"
    url += "/json?"
    # url += "maxAlternatives=1"
    url += "routeRepresentation=polyline"
    # url += "&avoid=tollRoads"
    url += "&travelMode=car"
    # url += "&vehicleMaxSpeed=130"
    # url += "&vehicleWeight=1700"
    url += "&vehicleCommercial=false"
    url += "&vehicleEngineType=electric"
    # url += "&consumptionInkWhPerkmAltitudeGain=2.2"
    # url += "&recuperationInkWhPerkmAltitudeLoss=1.5"
    # url += "&constantSpeedConsumptionInkWhPerHundredkm=90%2C13.7"
    # url += "&currentChargeInkWh=25"
    # url += "&maxChargeInkWh=60"
    # url += "&auxiliaryPowerInkW=1.7"
    url += f"&key={token}"
    for key, value in car.items():
        url+= f'&{key}={value}' 

    print(url)
    # Send the request to the TOMTOM API
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response into a Python dictionary
        data = response.json()
        return [route for route in data['routes']]
    else:
        print(f"Error retrieving data from OSRM API: {response.status_code}\n{response.content}")


def request_EV_route_tt_api(start,goal, car):
    headers = {
        'Accept': '*/*',
        # 'Authorization': '5b3ce3597851110001cf62487847161cf821486185bca1811e2819a6',
        'Content-Type': 'application/json'
    }

    token = os.environ.get('TOMTOM_TOKEN')
    url = "https://api.tomtom.com/routing/1/calculateLongDistanceEVRoute/"
    url += f"{start.point.y}%2C{start.point.x}%3A"
    url += f"{goal.point.y}%2C{goal.point.x}"
    url += "/json?"
    url += "routeRepresentation=polyline"
    url += "&maxAlternatives=3"
    url += "&avoid=tollRoads"
    # url += "routeType=fastest"
    url += "&travelMode=car"
    # url += "&vehicleMaxSpeed=130"
    # url += "&vehicleWeight=1700"
    # url += "&vehicleCommercial=false"
    url += "&vehicleEngineType=electric"
    # url += "&consumptionInkWhPerkmAltitudeGain=2.2"
    # url += "&recuperationInkWhPerkmAltitudeLoss=1.5"
    # url += "&constantSpeedConsumptionInkWhPerHundredkm=90%2C13.7"
    # url += "&currentChargeInkWh=25"
    # url += "&maxChargeInkWh=60"
    # url += "&auxiliaryPowerInkW=1.7"
    url += f"&key={token}"

    car_dict = car.__dict__.copy()
    _ = car_dict.pop('name')
    charge_specs = car_dict.pop('charge_specs')
    for key, value in car_dict.items():
        url+= f'&{key}={value}' 
    print(url)
    
    # Send the request to the TOMTOM API
    response = requests.post(url, json=charge_specs, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response into a Python dictionary
        data = response.json()
        return [route for route in data['routes']]
    else:
        print(f"Error retrieving data from TOMTOM API: {response.status_code}\n{response.content}")        


def request_osrm_api(start, goal):
        # Build the OSRM API URL
    url = f"http://router.project-osrm.org/route/v1/driving/"
    url += f"{start.point.x},{start.point.y};"
    url += f"{goal.point.x},{goal.point.y}?"
    url += f"overview=full&geometries=geojson&"
    url += "alternatives=3&steps=true"
    # Send the request to the OSRM API
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response into a Python dictionary
        data = response.json()
        return data
    else:
        print(f"Error retrieving data from TOMTOM API: {response.status_code}")


def request_openrouteservice_api(start, goal, types=1, preference=1):
    avoid = ["highways","tollways"]
    options = [ avoid[type] for type in types ] if types else []
    print(options)
    pref = ["fastest","shortest"]
    preference = pref[preference]
    print(preference)
    body = {
        "coordinates":[
            [start.point.x, start.point.y],
            [goal.point.x, goal.point.y]
        ],
        # "alternative_routes":{"target_count":3}, #{"share_factor":0.50,"target_count":3,"weight_factor":3.5},
        "attributes":["detourfactor"],
        # "extra_info":["tollways","waycategory"],
        "instructions":"false",
        "instructions_format":"text",
        "options":{"avoid_features": options},   #["tollways"]},
        "preference":preference                   #"recommended"
    }
    # print(body)
    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': '5b3ce3597851110001cf62487847161cf821486185bca1811e2819a6',
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post(
        'https://api.openrouteservice.org/v2/directions/driving-car/geojson', 
        json=body, 
        headers=headers
    )
    
    # Check if the request was successful
    if call.status_code == 200:
        # Parse the JSON response into a Python dictionary
        data = call.json()
        return Route(data, start, goal, api='ORS')
    else:
        print(f"Error retrieving data from OSRM API: {call.status_code}")

In [6]:
import urllib.parse

class Car:
    def __init__(
        self, name, avoid, vehiculeMaxSpeed,
        vehicleWeight, consumptionInkWhPerkmAltitudeGain, 
        recuperationInkWhPerkmAltitudeLoss, constantSpeedConsumptionInkWhPerHundredkm,
        currentChargeInkWh, maxChargeInkWh, auxiliaryPowerInkW,
         minChargeAtDestinationInkWh, minChargeAtChargingStopsInkWh,
        charge_specs):
        
        self.name = name
        self.avoid = avoid
        self.vehicleMaxSpeed = vehiculeMaxSpeed
        self.vehicleWeight = vehicleWeight
        self.consumptionInkWhPerkmAltitudeGain = consumptionInkWhPerkmAltitudeGain
        self.recuperationInkWhPerkmAltitudeLoss = recuperationInkWhPerkmAltitudeLoss
        self.constantSpeedConsumptionInkWhPerHundredkm = urllib.parse.quote(constantSpeedConsumptionInkWhPerHundredkm)
        self.currentChargeInkWh = currentChargeInkWh
        self.maxChargeInkWh = maxChargeInkWh
        self.auxiliaryPowerInkW = auxiliaryPowerInkW
        self.minChargeAtDestinationInkWh = minChargeAtDestinationInkWh
        self.minChargeAtChargingStopsInkWh = minChargeAtChargingStopsInkWh
        self.charge_specs = charge_specs


charge_specs = {
    "chargingModes": [
        {
        "chargingConnections": [
            {
            "facilityType": "Charge_380_to_480V_3_Phase_at_32A",
            "plugType": "IEC_62196_Type_2_Outlet"
            }
        ],
        "chargingCurve": [
            {
            "chargeInkWh": 6,
            "timeToChargeInSeconds": 360
            },
            {
            "chargeInkWh": 12,
            "timeToChargeInSeconds": 720
            },
            {
            "chargeInkWh": 28,
            "timeToChargeInSeconds": 1944
            },
            {
            "chargeInkWh": 40,
            "timeToChargeInSeconds": 4680
            }
        ]
        },
        {
        "chargingConnections": [
            {
            "facilityType": "Charge_200_to_240V_1_Phase_at_10A",
            "plugType": "Standard_Household_Country_Specific"
            }
        ],
        "chargingCurve": [
            {
            "chargeInkWh": 6,
            "timeToChargeInSeconds": 15624
            },
            {
            "chargeInkWh": 12,
            "timeToChargeInSeconds": 32652
            },
            {
            "chargeInkWh": 28,
            "timeToChargeInSeconds": 76248
            },
            {
            "chargeInkWh": 40,
            "timeToChargeInSeconds": 109080
            }
        ]
        }
    ]
    }

vroom = Car(
        name = 'vroum',
        avoid = 'tollRoads',
        vehiculeMaxSpeed= 130,
        vehicleWeight= 1700,
        consumptionInkWhPerkmAltitudeGain= 2.2,
        recuperationInkWhPerkmAltitudeLoss= 1.5,
        constantSpeedConsumptionInkWhPerHundredkm= '80,7.8:130,9.1',
        currentChargeInkWh= 30,
        maxChargeInkWh= 40,
        auxiliaryPowerInkW= 1.7,
        minChargeAtDestinationInkWh= 15,
        minChargeAtChargingStopsInkWh= 5,
        charge_specs= charge_specs
        )

In [7]:
# # Cell to debug tomtom calculateLongDistanceEVRoute

# tmp = request_EV_route_tt_api(start, goal, car, charge_specs)
# tmp


# routes = [Route(route,start,goal,api='TT') for route in tmp]
# len(routes)

# # routes[0].trace
# # routes[1].trace
# # len(routes[0]['legs']), len(routes[1]['legs'])

In [8]:
from shapely import Point
import polyline

# Define the charging station class
class ChargingStation:
    def __init__(self, feature_collection, idx=0):
        AddressInfo = feature_collection['AddressInfo']
        Connections = feature_collection['Connections']

        self.AddressInfo = AddressInfo # dict
        self.Connections = Connections # list of dicts

        self.id = feature_collection['ID']
        self.uuid = feature_collection['UUID']
        self.provider = feature_collection['DataProvider']['Title']
        self.address = AddressInfo['AddressLine1']
        self.point = Point(AddressInfo['Longitude'], AddressInfo['Latitude'])
        self.nbr_connections = len(Connections)
        try:
            self.level = Connections[idx]['Level']['IsFastChargeCapable'] 
        except:
            self.level = None
        try:
            self.power = Connections[idx]['PowerKW'] 
        except:
            self.power = 0
        try:
            self.type = Connections[idx]['CurrentType']['Title']
        except:
            self.type = None


import numpy as np
def line_split(line, num_splits):
    splits = np.array_split(np.arange(len(line.coords)), num_splits, axis=0)
    for split in splits:
        points = line.coords[split[0]:split[-1]]
        yield LineString(points)    

def remove_duplicate_stations(stations):
    keep = dict()
    for n, station in enumerate(stations):
        keep[station.id] = (n)

    return [stations[keep[key]] for key in keep.keys()]


def send_request(querystring, output=-1):
    url = "https://api.openchargemap.io/v3/poi"
    headers = {"Accept": "application/json"}
    
    with requests.Session() as s:
        response = s.get(url, headers=headers, params=querystring,)
    # print(response)

    if response.status_code == 200:
        # display(response.content)
        try:
            datas = response.json()
        except:
            datas = {}

        if datas != {}:    
            # list of available infos
            if output > -1:
                for n, data in enumerate(datas):
                    for key in data.keys():
                        if n == output : print(f'[{n}]-key {key} : {data[key]}')
            
            # list of charging stations
            charging_stations = []
            count = 0
            for n, data in enumerate(datas):
                plugs = len(data['Connections'])
                if output > -1: print(f'{n + 1} with {plugs} plugs / {len(datas)} stations')
                if data['DataQualityLevel'] >0:
                    # TODO handle data quality
                    for n in range(plugs):
                        count += 1
                        charging_stations.append(
                            ChargingStation( data, idx=n )
                        )
            # try:
            #     print(f'Number of total plugs: {count}')   
            # except:
            #     raise Exception('No charging stations returned')
            if len(charging_stations) > 0:
                return charging_stations 
            else:
                return []   #raise Exception('No valid charging stations found')
        else:
            return []     #raise Exception(response.text)


import requests
import time
def get_charging_stations(geom, maxresults=25, distance=2, output=-1, tolerance=0.01):
    url = "https://api.openchargemap.io/v3/poi"

    headers = {"Accept": "application/json"}

    querystring = {
        "maxresults": f"{maxresults}",
        "countrycode": "80",
        # "latitude": string
        # "longitude": string
        "distance": f"{distance}",
        "distanceunit": "km",
        # "opendata": "true",
        # "polyline": google polyline encoded: string
        "key": "42fc0eed-7a1d-49f6-b7a2-724b136adb3b"
        }
    try:        # is geom a location ?
        querystring['latitude'] = geom.point.y
        querystring['longitude'] = geom.point.x
        return send_request(querystring, output=output)
    except:     # geom is a road
        km_dist = lambda x : pyproj.Geod(ellps='WGS84').geometry_length(x) / 1e3
        chunks = km_dist(geom.trace) // 19 + 1
        print(f'Partitionning route in {int(chunks)} parts of 19 km',end='  ')

        stations = []
        for n, line in enumerate(line_split(geom.trace, chunks)):
            print(f'{n+1}',end='|')
            # print(f'portion {n+1} / {chunks}')
            simplified = line.simplify(tolerance=tolerance, preserve_topology=True)
            # inv_coords = LineString([(coords[1], coords[0]) for coords in simplified.coords])
            simplified_enc = polyline.encode(simplified.coords, precision=5, geojson=True) 
            querystring['polyline'] = f"{simplified_enc}" 
            stations += send_request(querystring, output=output)
            # time.sleep(0.2)
        print('\n')    
        return remove_duplicate_stations(stations)

   

def get_charge_info_on_TT_route(route):
    _charges = []
    
    for leg in route.legs[:-1]:
        _infos = leg['summary']
        _point = leg['points'][-1]
        _infos['chargingInformationAtEndOfLeg']['chargingParkLocation'].update(_point)
        _charges.append(_infos)

    return _charges        

# start_charging_stations = get_charging_stations(start, maxresults=500, distance=5, output=1)         

In [9]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MeasureControl
from shapely.geometry import mapping
import branca.colormap as cm
import pyproj


def make_map(start_city, goal_city, routes, charge_info):
    color_scale = cm.StepColormap(
        ['blue', 'green', 'yellow', 'orange', 'red', 'purple'], 
        vmin=0, vmax=5, index=[0, 1, 2, 3, 4, 5], caption='My Color Map'
)

    # Create a Folium map centered on the start city
    m = folium.Map(location=[start_city.point.y, start_city.point.x], crs='EPSG3857', zoom_start=10)
    

    group1 = folium.FeatureGroup(name='Area')
    # Add the start city polygon as a GeoJSON layer
    folium.GeoJson(mapping(start_city.area),
                   style_function=lambda x :{"fillColor": 'red',
                                             "fillOpacity": 0.09,
                                             "weight": 0.3,
                                             "color": 'black'},
                   name=start_city.name).add_to(group1)

    # Add the goal city polygon as a GeoJSON layer
    folium.GeoJson(mapping(goal_city.area), 
                   style_function=lambda x :{"fillColor": 'red',
                                             "fillOpacity": 0.09,
                                             "weight": 0.3,
                                             "color": 'black'},
                   name=goal_city.name
    ).add_to(group1)

    group1.add_to(m)

    for n, route in enumerate(routes):
        _duration = datetime.timedelta(seconds=route.duration)
        folium.GeoJson(mapping(route.trace), 
                    style_function=lambda x :{  #"fillColor": 'blue',
                                                # "fillOpacity": 0.09,
                                                "weight": 2.1,
                                                "color": color_scale(n)},
                    name=f'route_{n}',
                    tooltip=folium.Tooltip(f'Length: {route.distance / 1e3:.1f} km<br>Duration: {_duration}')
        ).add_to(m)
        
    
    # Trip charge stops
    for n, route in enumerate(routes):
        group2 = folium.FeatureGroup(name=f'Charge_{n}')
        for charge in charge_info[n]:
            info = charge[0]['chargingInformationAtEndOfLeg']
            _duration = datetime.timedelta(seconds=info['chargingTimeInSeconds'])        
            folium.Marker(
                                location=[
                                    info['chargingParkLocation']['latitude'], 
                                    info['chargingParkLocation']['longitude']
                                        ],
                                popup=f"Charge<br>Target: { info['targetChargeInkWh'] }kWh<br>Time: {_duration}",
                                icon=folium.Icon(color="green")
                            ).add_to(group2)
        group2.add_to(m)    

    group3 = folium.FeatureGroup(name='City Markers')
    # Add a marker for the start city
    folium.Marker(
        location=[start_city.point.y, start_city.point.x],
        popup=start_city.name
    ).add_to(group3)

    # Add a marker for the goal city
    folium.Marker(
        location=[goal_city.point.y, goal_city.point.x],
        popup=goal_city.name
    ).add_to(group3)

    group3.add_to(m)

    # Add a Measure Control to the map to display distances
    MeasureControl(position='bottomleft', active_color='red', completed_color='red').add_to(m)


    folium.LayerControl(position='topright').add_to(m)
    
    return  m


In [10]:
start = City(get_city_coords('MARSEILLE','France'))
# start_charging_stations = get_charging_stations(start, maxresults=1000, distance=5)
# print(f'Number of charging stations around start({start.point}) : {len(start_charging_stations)}')  


goal = City(get_city_coords('LILLE','France'))
# goal_charging_stations = get_charging_stations(goal, maxresults=1000, distance=5)
# print(f'Number of charging stations around goal({goal.point}) : {len(goal_charging_stations)}')  


def remove_duplicate_routes(routes, stations):
    keep = dict()
    for n, route in enumerate(routes):
        keep[route.trace] = (n)

    return [routes[keep[key]] for key in keep.keys()], [stations[keep[key]] for key in keep.keys()]

# routes = []
# charging_stations = []
# for type in [[],[0],[1]]:
#     for preference in [0,1]:
#         route = request_openrouteservice_api(start, goal, types=type, preference=preference)
#         routes.append(route)
#         stations = get_charging_stations(route, tolerance=0.001, maxresults=5000, distance=5)
#         charging_stations.append(stations)
#         print(f"route_OSRM length: {route.distance / 1e3}|Api", end=' | ')
#         print(pyproj.Geod(ellps='WGS84').geometry_length(route.trace) / 1e3, end=' | ')
#         print(f'Charge stations: {len(stations)}', end=' | ')
#         print(np.unique([ station.provider for station in stations], return_counts=True))
#         print('_' * 80)

# print(len(routes), len(charging_stations))
# routes_updated, charging_stations_updated = remove_duplicate_routes(routes, charging_stations)
# print(len(routes_updated), len(charging_stations_updated))

# routes_updated = [ Route(request_route_tt_api(start, goal, car), start, goal, api='TT') ]
routes = request_EV_route_tt_api(start, goal, vroom)
stats = [len(route['legs']) for route in routes]
print(f'Nombre de recharges par routes:{stats}')

routes_updated = [ RouteTT(route, start, goal) for route in routes]
charge_info = [ get_charge_info_on_TT_route(route) for route in routes_updated]
# charging_stations_updated = [ get_charging_stations(
#                         route, 
#                         tolerance=0.001, maxresults=5000, distance=5) for route in routes_updated ]

for route in routes_updated:
    print(f"route length: {route.distance / 1e3}|Api", end=' | ')
    print(pyproj.Geod(ellps='WGS84').geometry_length(route.trace) / 1e3, end=' | ')
    _duration = datetime.timedelta(seconds=route.duration)
    print(f' duration: {_duration}')


m = make_map(start, goal, routes_updated, charge_info)



# Display the map
display(m)

m.save(os.path.join('datas', 'TT_API_map.html'))
del m

https://api.tomtom.com/routing/1/calculateLongDistanceEVRoute/43.292404113171244%2C5.419087955946399%3A50.63184432452358%2C3.047001435221407/json?routeRepresentation=polyline&maxAlternatives=3&avoid=tollRoads&travelMode=car&vehicleEngineType=electric&key=yjFXQX1HCdQjGQAmSOgf013L8bwaqify&avoid=tollRoads&vehicleMaxSpeed=130&vehicleWeight=1700&consumptionInkWhPerkmAltitudeGain=2.2&recuperationInkWhPerkmAltitudeLoss=1.5&constantSpeedConsumptionInkWhPerHundredkm=80%2C7.8%3A130%2C9.1&currentChargeInkWh=30&maxChargeInkWh=40&auxiliaryPowerInkW=1.7&minChargeAtDestinationInkWh=15&minChargeAtChargingStopsInkWh=5
Nombre de recharges par routes:[7, 7, 7, 7]
route length: 1144.038|Api | 1144.0341328107245 |  duration: 16:42:27
route length: 1059.622|Api | 1059.617459793365 |  duration: 17:24:04
route length: 1100.462|Api | 1100.4621857880327 |  duration: 17:49:47
route length: 1097.352|Api | 1097.3770420853098 |  duration: 18:03:32


In [13]:
import pandas as pd

def pimp(df):
    df['departureTime'] = pd.to_datetime(results[0]['departureTime'])
    df['arrivalTime'] = pd.to_datetime(results[0]['arrivalTime'])
    df['charge_time'] = df.departureTime - df.shift(1).arrivalTime
    charge_info = df.pop('chargingInformationAtEndOfLeg')
    
    return pd.concat([df, pd.json_normalize(charge_info)], axis=1)


results = []
for route in routes_updated:
    dfs=[]
    for leg in route.legs:
        # display(leg)
        # display(leg['points'][-1], leg['summary'])
        dfs.append(
                        pd.concat(
                            [
                                pd.DataFrame.from_records([leg['points'][-1]]), 
                                pd.DataFrame.from_records([leg['summary']])
                            ], axis=1
                            )
                        
                )
    results.append( pd.concat(dfs, axis=0).reset_index(drop=True)   ) # break
    # break

results = [pimp(result) for result in results]  

results[0].T

,0,1,2,3,4,5,6
latitude,43.6515,44.72179,46.1046,47.4113,48.86028,50.26655,50.63157
longitude,3.51363,3.28488,3.19622,2.92598,2.67226,2.79693,3.04655
lengthInMeters,199766,167356,181595,175275,180145,184812,55089
travelTimeInSeconds,8915,6201,6588,6858,8817,9215,2333
trafficDelayInSeconds,0,0,0,119,0,0,0
trafficLengthInMeters,0,0,0,10780,0,0,0
departureTime,2023-05-01 08:17:58+02:00,2023-05-01 11:17:05+02:00,2023-05-01 13:32:32+02:00,2023-05-01 15:53:48+02:00,2023-05-01 18:19:57+02:00,2023-05-01 21:19:45+02:00,2023-05-02 00:21:29+02:00
arrivalTime,2023-05-01 10:46:33+02:00,2023-05-01 13:00:25+02:00,2023-05-01 15:22:20+02:00,2023-05-01 17:48:06+02:00,2023-05-01 20:46:54+02:00,2023-05-01 23:53:19+02:00,2023-05-02 01:00:22+02:00
batteryConsumptionInkWh,21.139898,20.594379,18.626062,18.233605,18.684045,19.571955,5.747063
remainingChargeAtArrivalInkWh,8.860102,5.055621,5.273938,5.316395,5.215955,5.028045,15.041494


----------------------- DEV ZONE ----------------------------------------